## To-do list
* Some cohort exploration based on deletion of subjects

* Look at variable correlations & clustering (if possible) -- suspect eg that systolic/diastolic will cluster based on prev work & means we can get away with using a variable score or one but not the other

* Model checking!!

**Noted:**
* labs need transformation
* meds can't be used; too sparse (though try exposure variables)
* everything else should be good
* use impute.ipynb to do variable reduction/selection

## For a general modeling pipeline:
Need a utility function that:
1. ~~Takes a DF and a list of variables,~~
2. ~~Makes a GLM and samples from it,~~
3. ~~Returns the trace & the model.~~

This will create a bunch of models & traces to check model diagnostics on, LOO, WAIC, etc & pick some "good" models.

Then need utility functions for:
1. Plotting chosen diagnostics.
2. Doing model comparisons using WAIC/LOO.
3. Measuring the accuracy vs validation set -- this will require refitting the model & using sample_ppc w/ shared theano variables to predict off the validation set.

FINALLY, take the best model that pops out of the above & refit it with both train+validation sets, then use it to predict the test set as final check of accuracy.

In [15]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm
import re
sns.set()

from sklearn import preprocessing, model_selection
from sklearn.metrics import make_scorer, confusion_matrix, f1_score, roc_auc_score

In [2]:
import warnings
warnings.filterwarnings("ignore", module="mkl_fft")
warnings.filterwarnings("ignore", module="matplotlib")

In [3]:
df = pd.read_csv('../data/merged.csv', parse_dates=['admit_date','discharge_date','dob','dod'])

In [4]:
df.head()

,ruid,visit_id,admit_date,discharge_date,stay_length,n_transfers,readmit_time,readmit_30d,sex,dob,...,lab_diastolic_median,lab_systolic_5p,lab_diastolic_5p,lab_systolic_95p,lab_diastolic_95p,lab_systolic_std,lab_diastolic_std,age,total_encounters,group
0,50135262,0,2007-02-08,2007-02-12,4,2,172 days 00:00:00.000000000,0,F,1949-09-20,...,58.0,107.0,44.0,186.0,77.0,20.263715,9.868120,57.385352,10,train
1,50135262,1,2007-08-03,2007-08-06,3,3,22 days 00:00:00.000000000,1,F,1949-09-20,...,61.0,106.0,50.0,146.0,74.0,12.518349,8.314473,57.867214,10,train
2,50135262,2,2007-08-28,2007-08-29,1,1,179 days 00:00:00.000000000,0,F,1949-09-20,...,60.0,108.0,42.0,188.0,82.0,25.989675,10.976472,57.935661,10,train
3,50135262,3,2008-02-24,2008-02-28,4,2,44 days 00:00:00.000000000,0,F,1949-09-20,...,74.0,116.0,59.9,165.0,88.0,15.661763,7.936733,58.428474,10,train
4,50135262,4,2008-04-12,2008-04-13,1,1,928 days 00:00:00.000000000,0,F,1949-09-20,...,66.0,122.0,56.0,158.0,88.0,11.855141,11.269165,58.559890,10,train


In [5]:
list(df.columns)

['ruid',
 'visit_id',
 'admit_date',
 'discharge_date',
 'stay_length',
 'n_transfers',
 'readmit_time',
 'readmit_30d',
 'sex',
 'dob',
 'dod',
 'race',
 'cpt_anesthesia',
 'cpt_eval_manage',
 'cpt_expired',
 'cpt_medicine',
 'cpt_modifier',
 'cpt_path_lab',
 'cpt_radiology',
 'cpt_surgery',
 'cpt_unknown',
 'icd_dx_blood',
 'icd_dx_circulatory',
 'icd_dx_congenital',
 'icd_dx_digestive',
 'icd_dx_endocrine',
 'icd_dx_external',
 'icd_dx_gu',
 'icd_dx_infection',
 'icd_dx_injury',
 'icd_dx_mental',
 'icd_dx_muscskel',
 'icd_dx_neoplasm',
 'icd_dx_nervous',
 'icd_dx_obstetric',
 'icd_dx_perinatal',
 'icd_dx_respiratory',
 'icd_dx_skin',
 'icd_dx_symptoms',
 'icd_proc',
 'icd_visit',
 'med_Antihypertensive Agents',
 'med_Analgesics, Opioid',
 'med_Narcotics',
 'med_Antipyretics',
 'med_Anti-Bacterial Agents',
 'med_Anti-Inflammatory Agents, Non-Steroidal',
 'med_Analgesics, Non-Narcotic',
 'med_Antiemetics',
 'med_Diuretics',
 'med_Anti-Arrhythmia Agents',
 'med_Anti-Allergic Agents',
 

In [11]:
df.sex = (df.sex == 'M')*1
df.drop('race',axis=1, inplace = True) # can't use this; there's just too few of some races for it to be meaningful

In [12]:
df.columns = list(df.columns.str.replace(" |, ","_").str.replace("-","_").str.lower())
# processing the med names so they play nicely with patsy/glm

In [13]:
id_vars = list(df.columns[0:3]) + ['total_encounters','group','dob','dod']
meds = list(df.columns[df.columns.str.contains('med_')])
cpts = list(df.columns[df.columns.str.contains('cpt_')])
dx = list(df.columns[df.columns.str.contains('icd_dx')])
labs = list(df.columns[df.columns.str.contains('lab_')])
bmi = list(df.columns[df.columns.str.contains('bmi|pregnancy')])
demos = ['age','sex']
outcomes = ['readmit_time','readmit_30d']
visit = cpts + ['n_transfers','stay_length','icd_proc','icd_visit']
final_var = demos + bmi + dx + labs + visit # drop meds; can't use them right off the bat

In [22]:
# select various different lab measures -- using all of them is redundant
labs_median = list(df.columns[df.columns.str.contains('lab_.*_median')])
labs_last = list(df.columns[df.columns.str.contains('lab_.*_last')]) + ['lab_systolic_median','lab_diastolic_median']
labs_extremes = list(df.columns[df.columns.str.contains('lab_.*5p')])
labs_var = list(df.columns[df.columns.str.contains('lab_.*_std')]) # redundancy analysis suggests this contains the most information not included in other variables

In [60]:
# meds_desc = df[meds].describe()
# visit_desc = df[visit].describe()
# dx_desc = df[dx].describe()
# labs_desc = df[labs].describe()
# demos_desc = df[demos].describe()

In [23]:
X = df[['readmit_30d'] + final_var + ['group']]

In [26]:
train = X[X.group=='train'].copy()
train.drop(columns='group', inplace=True)

valid = X[X.group=='valid'].copy()
valid.drop(columns='group', inplace=True)

test = X[X.group=='test'].copy()
test.drop(columns='group', inplace=True)

assert(X.shape[0]==(train.shape[0] + valid.shape[0] + test.shape[0]))

In [27]:
trans_var = ['age'] + labs + ['bmi_last']
scaler = preprocessing.StandardScaler()

scaler.fit(train[trans_var]) # recommended you train your scaler only on a training split, not validate or test split

train[trans_var] = scaler.transform(train[trans_var])
valid[trans_var] = scaler.transform(valid[trans_var])
test[trans_var] = scaler.transform(test[trans_var])

# some variables need to be transformed to work well in the glm

In [72]:
# train_red = train.dropna()

In [104]:
def model_gen(outcome = 'readmit_30d', variables = final_var, data = train, draws = 1000):
    import pymc3 as pm
    import pymc3.glm as glm
    
    formula = outcome + ' ~ ' + ' + '.join(variables)
    family = pm.glm.families.Binomial()
    
    with pm.Model() as model:
        glm.GLM.from_formula(formula,data,family = family)
    
        start = pm.find_MAP() # need to start at the MAP because otherwise the initial energies can cause the chains to fail
        
        trace = pm.sample(draws, start = start)
        
    return model, trace

In [ ]:
# model diagnostics
def model_diagnostics():
    pass

In [ ]:
# model validation
def validate_model():
    # in here we're going to use ppc to draw samples from the posterior vs data in the validation set
    # and then we'll see how well our model does
    pass